In [ ]:
import tensorflow as tf
import keras

loaded_model = tf.keras.models.load_model('C:/Users/ehemingway/Downloads/drive-download-20230110T185119Z-001/Model_Distortion_Coefficient_320x340_3x3_MSE_128_20230105')
loaded_model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(loaded_model,to_file='model.png')

In [ ]:
from keras.models import Model

inp = loaded_model.input                                  # input placeholder
out = [layer.output for layer in loaded_model.layers]            # all layer outputs
activation_model = Model(inputs=loaded_model.input, outputs=out)

from PIL import Image
import numpy as np

imName = '0075_19'
input_image = Image.open('Training Set GS 1000 320x240/'+imName+'.jpg')
x = np.array(input_image)
x = np.expand_dims(x,axis=0)
x = x / 255.

input_data = x
activations = activation_model.predict(input_data)

# Print the output of each layer
for i, activation in enumerate(activations):
    print("Layer {} output shape: {}".format(i, activation.shape))


In [ ]:
output_of_layer_0 = activations[0]
output_of_layer_1 = activations[1]
output_of_layer_2 = activations[2]
output_of_layer_3 = activations[3]
output_of_layer_4 = activations[4]
import matplotlib.pyplot as plt
import os
curDir = os.getcwd()
lowerDir = 'Convolutional First Layer Feature Maps for '+imName
path = os.path.join(curDir,lowerDir)
os.mkdir(path)
for i in range(32):
    img = Image.fromarray(output_of_layer_0[0,:,:,i]*255)
    img = img.convert('L')
    img = img.resize((1280,960))
    img.save(path+'/'+str(i)+'.jpg')

lowerDir = 'Convolutional Second Layer Feature Maps for '+imName
path = os.path.join(curDir,lowerDir)
os.mkdir(path)
for i in range(64):
    img = Image.fromarray(output_of_layer_2[0,:,:,i]*255)
    img = img.convert('L')
    img = img.resize((1280,960))
    img.save(path+'/'+str(i)+'.jpg')

lowerDir = 'Convolutional Third Layer Feature Maps for '+imName
path = os.path.join(curDir,lowerDir)
os.mkdir(path)
for i in range(64):
    img = Image.fromarray(output_of_layer_4[0,:,:,i]*255)
    img = img.convert('L')
    img = img.resize((1280,960))
    img.save(path+'/'+str(i)+'.jpg')

In [ ]:
output_of_layer_9 = activations[9]
k = -output_of_layer_9

import scipy

xSuckInt = np.array([0]*320*240).reshape(240,320).astype(np.uint8)
ySuckInt = np.array([0]*320*240).reshape(240,320).astype(np.uint8)
c = 0
for xOut in range(-160,160):
    r = 0
    for yOut in range(-120,120):
        rOut = (xOut**2+yOut**2)**0.5/(120**2+160**2)**0.5
        theta = np.arctan2(yOut,xOut)
        rSuck = rOut*(1+k*rOut**2)
        xSuck = rSuck*np.cos(theta)
        ySuck = rSuck*np.sin(theta)
        xSuckInt[r,c]=xSuck*160
        ySuckInt[r,c]=ySuck*120
        r = r + 1
    c = c + 1

undistortedIm = scipy.interpolate.interp2d(xSuckInt,ySuckInt,input_data[0,:,:])


In [ ]:
imageUndistort = Image.fromarray(np.uint8(undistortedIm(xSuckInt.reshape(-1),ySuckInt.reshape(-1))*255),'L')